detectron2 using cocolike structure training mask rcnn for 5000 iterations.

In [1]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [1]:
#!pip install -q -U pyyaml==5.1 'pycocotools>=2.0.1'
#!pip install -q -U torch==1.6.0+cu102 torchvision==0.7.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install -q -U pycocotools --use-feature=2020-resolver

import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)
#test

1.6.0 False 10.1


In [ ]:
# only for linux.
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [2]:
import os
import copy
import json
import pycocotools
import detectron2
import random

import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")

import detectron2
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

from detectron2.data import MetadataCatalog,DatasetMapper,build_detection_train_loader,build_detection_test_loader
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 

#from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.projects.deeplab import add_deeplab_config, build_lr_scheduler

In [6]:
PATH = os.path.abspath(os.getcwd())

register_coco_instances("my_dataset_train1",{},PATH + "/input/train_annotations.json",PATH + "/input/train_v2/") #TRAIN annotations got mixed up -.-
register_coco_instances("my_dataset_val",{},PATH + "/input/test_annotations_slim.json",PATH + "/input/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train1")
dataset_dicts = DatasetCatalog.get("my_dataset_train1")

AssertionError: Dataset 'my_dataset_train1' is already registered!

In [7]:
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train1")
dataset_dicts = DatasetCatalog.get("my_dataset_train1")

[11/21 10:26:44 d2.data.datasets.coco]: Loading /application/input/train_annotations.json takes 1.44 seconds.
[11/21 10:26:44 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json


In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train1",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.TEST.EVAL_PERIOD = 500
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 64
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)

In [9]:
def build_sem_seg_train_aug(cfg):
    augs = [T.AugmentationList([
        T.RandomBrightness(0.9, 1.1),
        T.RandomFlip(prob=0.5),
        T.RandomCrop("absolute", (128, 128))
        ])]
    return augs

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=build_sem_seg_train_aug(cfg))
        return build_detection_train_loader(cfg, mapper=mapper)
    
    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer):
        return build_lr_scheduler(cfg, optimizer)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train() #Trainer will throw out non-annotated pictures. 

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Should build valuator inside the training. So it could be evaluated from there. 

In [ ]:
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="../output/") #Should bne val but ffs.  
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator)) #https://medium.com/@yanfengliux/the-confusing-metrics-of-ap-and-map-for-object-detection-3113ba0386ef

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer

checkpointer = DetectionCheckpointer(trainer.model, save_dir="./")
checkpointer.save("model_mask_resnet101_rcnn")  # save to save_dir

In [ ]:
#https://www.kaggle.com/ptaneja/detectron2-notebook
import glob

def create_test_datatset():    
    img_dir = '/input/airbus-ship-detection/test_v2/'
    dataset_dicts = []
    
    for img_path in glob.glob(img_dir + '*.jpg'):
        record = {}
        file_path = img_path
        image_id = img_path.split('/')[-1].split('.')[0]
        record['file_name'] = file_path
        record['image_id'] = image_id
        dataset_dicts.append(record)
    return dataset_dicts

test_dataset = create_test_datatset()
img_ids = []
pred_string = []

DatasetCatalog.register("submit_test1", create_test_datatset)
od_dataset = MetadataCatalog.get("submit_test1")

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("submit_test1", )
predictor = DefaultPredictor(cfg)